In [1]:
from copy import deepcopy                        # Will be used to deep copy
import random                                    # Will be used for randomization
from math import exp                             # will be used for probability calculation
import numpy as np
import time

In [2]:
Courses = ["C1", "C2", "C3", "C4", "C5"]
Halls = ["H1", "H2"]
Slots = ["T1", "T2", "T3"]

# MaxHallHour = random.randint(6,9) #different for every hall
CourseHour = random.randint(1,3)#Same for every course
PopulationSize=100# number of individials in a population

CrossProb = 0.8   # probability for crossover
MutationProb = 0.1  # probability for mutation
TournamentSize = 5  #Tournament size

In [3]:
Conflict = []

#calculating conflict for each course. is random
for x in range(0, len(Courses)-1):
    for y in range(x+1, len(Courses)):
        r1 = random.randint(0,10)  # generate random value from  0 to 10
        if r1 >= 5:   #if >= 5, Conflict exists
            r2 = random.randint(1,10) #random number for number of conflicting students from 1 to 10   
            Conflict.append((Courses[x],Courses[y], r2))   #append it in conflicts



In [4]:
#Different for each hall based with range (6 to 9)
MaxHallHours = [];
for x in range(0, len(Halls)):
    MaxHallHours.append(random.randint(6,9));


In [5]:
#generates random solution
def Solution():
    solution = []
    for course in Courses:
        slot = random.choice(Slots)
        hall = random.choice(Halls)
        solution.append((course, slot, hall))
    return solution

In [6]:
#generates population based on size input
def Populate(Size):
    return [Solution() for _ in range(Size)]

In [7]:
# Population = Populate(PopulationSize)
# Population

In [8]:
#calculating fitness of solution
def Calculate_Fitness(Solution):
    PenaltyForHall = 10
    PenaltyForStudent = 100
    Score = 0 #Score to return
    Student = 0
    for x in range(0, len(Halls)):
        
        
        # calculate the total use for a particular hall
        Use = sum(CourseHour for S in Solution if S[2] == Halls[x])
        
        #find the max hour limit for that hall
        Max_Hour = MaxHallHours[x];
        
#         print(f"Use: {Use} Max: {Max_Hour}")
        
        #if greater, penalize with adding 10 for every hour exceeded
        if Use > Max_Hour:
            Score += (Use-Max_Hour) * PenaltyForHall
        
    #calculate penalty for similar conflict courses
    for (c1, c2, n) in Conflict:

        #if conflicting courses done at the same time in different hall, add the number of students
        if any(r1[0] == c1 and r2[0] == c2 and r1[1] == r2[1] for r1 in Solution for r2 in Solution):
            Student += n

#     print("Student: ",Student)
#     print("Score: ",Score)
    #multiply with penalty for student
    Student *=PenaltyForStudent
    
    #add it to the score
    Score += Student
                

    
    return Score

In [9]:
#calculating the fitness of each solution and returning it in an List
def Generate_Fitness(Population):
    Fitness = []
    for x in Population:
        fit = Calculate_Fitness(x)
        Fitness.append(fit)    
    
    return Fitness
    

In [10]:
# Fitness

In [11]:
#Single point crossover with probability defined at top
def Single_Crossover(P1, P2):
    if random.random() < CrossProb:
        CrossPoint = random.randint(1, len(P1) - 1)
        C1 = P1[:CrossPoint] + P2[CrossPoint:]
        C2 = P2[:CrossPoint] + P1[CrossPoint:]
        return C1, C2
    else:
        return P1, P2

In [12]:
#mutation with probability defined at top
def Mutation(child):
    for x in range(len(child)):
        if random.random() < MutationProb:
            course = child[x][0]
            slot = random.choice(Slots)
            hall = random.choice(Halls)
            child[x] = (course, slot, hall)
    return child

In [13]:
SolutionSet = []
#function to check for solution and store in SolutionSet
def Check_For_Solution(Population, Fitness):
    for x in range(0,len(Fitness)):
        if Fitness[x] == 0: #if fitness = 0
            SolutionSet.append(Population[x])  #Append in SolutionSet
#             del Population[x]  #delete it from the Population
#             del Fitness[x]
    return Population

In [14]:
def Tournament_List(Population, Fitness):
    List = []
    for x in range(0,TournamentSize):
        minval = min(Fitness)
        minindex = Fitness.index(minval)
        while minval == 0:
            minval = min(Fitness)
            minindex = Fitness.index(minval)
            del Fitness[minindex]
            del Population[minindex]


        
        
        List.append(Population[minindex])

    return List

In [15]:
#Function to apply crossover and get the child list using single crossover
def Apply_Crossover(TournamentList):
    ChildList = []
    for x in range(0, len(TournamentList)-1):
        for y in range(x+1, len(TournamentList)):
            p1 = TournamentList[x] #get parents
            p2 = TournamentList[y]
            
            (c1,c2) = Single_Crossover(p1,p2)  # get children
            if c1 != p1 and c2 != p2:
                ChildList.append(c1)  # append them to childlist
                ChildList.append(c2)
    
    return ChildList #return child list
            

In [16]:
def Apply_Mutation(ChildList):
    MutatedList = []
    
    for x in ChildList:
        ML = Mutation(x)
        MutatedList.append(x)
    
    return MutatedList

In [17]:
counter = 0
Population = Populate(PopulationSize) # randomly populate
start = time.time()
while counter < PopulationSize:   # while Populationsize not eached
    Fitness = Generate_Fitness(Population)   # generate fitness
    Population = Check_For_Solution(Population, Fitness)   #check for solutions present
    
    TournamentList = Tournament_List(Population,Fitness)  # select the best from tournament selection
    
    ChildrenList = Apply_Crossover(TournamentList) # apply crossover and get the children list
    
    MutatedList = Apply_Mutation(ChildrenList)  # mutate the children list
    
    TournamentList.extend(MutatedList)   # add it to the tournament list
    
    Population = TournamentList    # the new population is the tournament list
    
    counter = counter+1
end = time.time()
print("Time Taken: ", end-start)
# SolutionSet

Time Taken:  0.023935556411743164


In [18]:
# print("Length of Population Set: ", len(Population))

In [19]:
print("Fitness: ", Fitness)

Fitness:  [100, 100, 100, 100, 100]


In [20]:
print("Length of Solution Set: ", len(SolutionSet))

print("Solution Set: ")
SolutionSet

Length of Solution Set:  8
Solution Set: 


[[('C1', 'T1', 'H2'),
  ('C2', 'T3', 'H1'),
  ('C3', 'T1', 'H1'),
  ('C4', 'T2', 'H2'),
  ('C5', 'T2', 'H2')],
 [('C1', 'T2', 'H1'),
  ('C2', 'T1', 'H2'),
  ('C3', 'T2', 'H2'),
  ('C4', 'T2', 'H2'),
  ('C5', 'T3', 'H1')],
 [('C1', 'T1', 'H1'),
  ('C2', 'T3', 'H2'),
  ('C3', 'T2', 'H2'),
  ('C4', 'T2', 'H1'),
  ('C5', 'T1', 'H1')],
 [('C1', 'T1', 'H1'),
  ('C2', 'T2', 'H1'),
  ('C3', 'T3', 'H2'),
  ('C4', 'T1', 'H1'),
  ('C5', 'T1', 'H2')],
 [('C1', 'T3', 'H1'),
  ('C2', 'T1', 'H2'),
  ('C3', 'T3', 'H2'),
  ('C4', 'T3', 'H2'),
  ('C5', 'T2', 'H2')],
 [('C1', 'T3', 'H1'),
  ('C2', 'T1', 'H1'),
  ('C3', 'T2', 'H1'),
  ('C4', 'T2', 'H2'),
  ('C5', 'T3', 'H1')],
 [('C1', 'T3', 'H1'),
  ('C2', 'T1', 'H2'),
  ('C3', 'T3', 'H1'),
  ('C4', 'T2', 'H1'),
  ('C5', 'T2', 'H2')],
 [('C1', 'T2', 'H2'),
  ('C2', 'T1', 'H1'),
  ('C3', 'T2', 'H1'),
  ('C4', 'T3', 'H1'),
  ('C5', 'T3', 'H2')]]

In [ ]:
#printing one of the solutions as optimal solution(the last one)
if SolutionSet:
    print("An Optimal Solution: ")
    print(SolutionSet[len(SolutionSet)-1])    


In [ ]:
print("Conflicts: ")
Conflict

In [ ]:
MaxHallHours


In [ ]:
CourseHour